In [ ]:
!pip install openpyxl
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
import os
import gc
import gensim

import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
!pip install transformers
import transformers
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split


# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

##0.1] Training are first 2 folds test is 3rd Fold

import base64
import requests
import pandas as pd
import os
import nltk


In [ ]:
import base64
import requests
import pandas as pd
import os
import nltk

##0.2] Load pretrained model

from simpletransformers.classification import ClassificationModel
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
# define hyperparameter
'''
"use_multiprocessing":False,
"use_multiprocessing_for_evaluation":False,
Those 2 were added for the 0% red bar thing
'''
train_args ={"reprocess_input_data": True,
             "use_multiprocessing":False,
             "use_multiprocessing_for_evaluation":False,
             "overwrite_output_dir": True,
             "fp16":False,
            "evaluate_during_training": True,
             "num_train_epochs": 4} 

# Create a ClassificationModel
model_en = ClassificationModel(
    #"roberta", 
    #"roberta-base",
    #"xlmroberta", 
    #"xlm-roberta-base",
    #"bert", 
    #"bert-base-cased",
    "electra",
    "google/electra-base-discriminator",
    args=train_args
)



In [ ]:
data_url = 'https://raw.githubusercontent.com/shammur/SemEval2022Task3/main/data/train/train_subtask-1/en/'
folds = [pd.read_csv(os.path.join(data_url, 'En-Subtask1-fold_{}.tsv'.format(i)), sep='\t') \
         for i in range(3)] #reading all folds from GitHub

df_en = pd.concat(folds[0:3]) #concatinate all folds in 1 fold (only 2 folds because 3rd used as test)
print(df_en)
print(len(df_en))

##1.0] Additional Preprocessing 8-12-2021

###1.1] TF-IDF : extract 2 entities

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
ngram=1
# #converting to lower case
df_en2=df_en
df_en2['Sentence'] = df_en2.Sentence.map(lambda x: x.lower()) 
#Removing the punctuation
df_en2['Sentence'] = df_en2.Sentence.str.replace('[^\w\s]', '')  

#splitting the word into tokens
#man_train_set_df['Sentence'] = man_train_set_df['Sentence'].apply(tokenize.word_tokenize) 
en_texts=df_en2['Sentence']

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=True, ngram_range=(1, ngram), norm='l1', 
                                   use_idf=True, max_df=0.05)#, min_df=3)# max_features=1000) 

en_feat = tfidf_vectorizer.fit_transform(en_texts)

en_feat.shape

TFIDF_Dict=[word for word in set([ell for lis in [el.split() for el in en_texts.tolist()] for ell in lis]) if word not in tfidf_vectorizer.get_feature_names()]

TFIDF_Dict

TFIDF_vocab=dict(zip(tfidf_vectorizer.get_feature_names(), en_feat.toarray()[0])) #key= word, value=vector (cat, dog)

TFIDF_vocab=[w for w in TFIDF_vocab]
TFIDF_vocab

def generateNewSentence(df):
  print(df['Sentence'])
  concat=[w for w in df['Sentence'].split() if w not in TFIDF_Dict]
  print('concat',concat)
  return df['Sentence']+". "+" ".join(concat)

#df_en['NewSentences'] = df_en.apply(generateNewSentence, axis=1)

df_en = pd.concat(folds[0:3]) #concatinate all folds in 1 fold (only 2 folds because 3rd used as test)


df_en

In [ ]:
#

In [ ]:
#Split dataset only 30% patterns in test but not train

len(df_en)

df_en=df_en.sort_values(['ID'])

df_en

test_df_patterns=pd.concat([df_en[1:451], df_en[455:508],df_en[511:701], df_en[2657:3291], df_en[32977:3526], df_en[4330:4508],df_en[5393:5437], df_en[5615:]])

test_df_patterns

In [ ]:
len(test_df_patterns)

new_df=df_en[~df_en.ID.isin(test_df_patterns.ID)]

new_df

In [ ]:
##2.1] Train

len(df_en)

left_test_to_be_added= len(df_en)*0.7 - len(test_df_patterns) 

left_test_to_be_added

percentage_test= left_test_to_be_added * 100/ len(new_df) 

percentage_test /100

df_en_train, df_en_test = train_test_split(new_df, test_size=percentage_test /100, shuffle=True)


df_en_test=pd.concat([df_en_test,test_df_patterns])

del df_en_train['ID'] #remove ID col
del df_en_test['ID'] #remove ID col

#df_en_train=pd.concat([df_en_train,WWM_df])

len(df_en_test)


if 'NewSentences' in df_en:
    df_en_test['Sentence']=df_en_test['NewSentences']
    df_en_train['Sentence']=df_en_train['NewSentences']

    del df_en_train['NewSentences'] #remove NewSentences
    del df_en_test['NewSentences'] #remove NewSentences

len(df_en_train)

df_en_test.head()

df_en_train.head()

In [ ]:
df_en_train, eval_df = train_test_split(df_en_train, test_size=0.1, shuffle=True, stratify=df_en_train['Labels'])
df_en_train

In [ ]:
eval_df

**CommonSense MultiTask Fine-tuning**

In [ ]:
df_common_sense=pd.read_excel(r'../input/common-sense/df_commonSenseValidation.xlsx',index_col=0)
df_common_sense[:300]

In [ ]:
df_common_sense=df_common_sense.rename(columns={"sentence": "Sentence", "label": "Labels"})
df_common_sense

## CommonSense Filtering

In [ ]:
#filter on based on words which are in TF-IDF vocab
TFIDF_sentences=[]
TFIDF_labels=[]
for sentence,label in zip(df_common_sense['Sentence'],df_common_sense['Labels']):
    if any(word in sentence for word in TFIDF_vocab):
        #print('yes',sentence,label)
        TFIDF_sentences.append(sentence)
        TFIDF_labels.append(label)


In [ ]:
commonSense_filtered_df=pd.DataFrame(list(zip(TFIDF_sentences,TFIDF_labels)),  columns =['Sentence', 'Labels'])


commonSense_filtered_df

In [ ]:
#df_en_train=pd.concat([df_en_train,commonSense_filtered_df])
df_en_train

In [ ]:
model_en.train_model(
        commonSense_filtered_df,
        eval_df=eval_df,
        accuracy=lambda truth, predictions: accuracy_score(
            truth, [round(p) for p in predictions]
        ),
)

In [ ]:
# Train the model
#model_en.train_model(commonSense_filtered_df)#, learning_rate=3e-4)

In [ ]:
### 2.2] Test : English

df_en_test

result, model_outputs, wrong_predictions = model_en.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

result

In [ ]:


len(model_outputs)

In [ ]:
[model_outputs]

In [ ]:
y_dev_pred = [int(el[1] > el[0]) for el in model_outputs]
df_en_test['y_dev_pred'] = y_dev_pred 
df_en_test[(df_en_test['Labels'] == 1) & (df_en_test['y_dev_pred'] == 1)]

In [ ]:
df_en_testWrong=df_en_test[(df_en_test['Labels'] != df_en_test['y_dev_pred'])]
df_en_testWrong

In [ ]:
count=0
for s in df_en_testWrong['Sentence']:
    #print(s)
    if 'specifically' in s:
        count+=1
print(count)

In [ ]:
#!pip install openpyxl

#df_en_test.to_excel('df_test_roberta.xlsx') 

# Train on original training set

In [ ]:
sum(df_en_train['Sentence'].str.count("specifically"))

In [ ]:
sum(df_en_test['Sentence'].str.count("specifically"))

In [ ]:
model_en.train_model(
        df_en_train,
        eval_df=eval_df,
        accuracy=lambda truth, predictions: accuracy_score(
            truth, [round(p) for p in predictions]
        ),
)

In [ ]:
#model_en.train_model(df_en_train)#, learning_rate=3e-5)

In [ ]:
#df_en_test['Sentence'] = df_en_test['Sentence'].str.replace('more specifically','in particular')

In [ ]:
df_en_test

In [ ]:
##2.2] Test : English

df_en_test

result2, model_outputs2, wrong_predictions2 = model_en.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

result2

In [ ]:
y_dev_pred = [int(el[1] > el[0]) for el in model_outputs2]
df_en_test['y_dev_pred'] = y_dev_pred 

df_en_testCorrect=df_en_test[(df_en_test['Labels'] == df_en_test['y_dev_pred'])]
df_en_testCorrect
count=0
for s in df_en_testCorrect['Sentence']:
    #print(s)
    if 'specfically' in s:
        count+=1
print(count)

In [ ]:
y_dev_pred = [int(el[1] > el[0]) for el in model_outputs2]
df_en_test['y_dev_pred'] = y_dev_pred 
df_en_test[(df_en_test['Labels'] == 1) & (df_en_test['y_dev_pred'] == 1)]
df_en_testWrong=df_en_test[(df_en_test['Labels'] != df_en_test['y_dev_pred'])]
df_en_testWrong
count=0
for s in df_en_testWrong['Sentence']:
    #print(s)
    if 'specifically' in s:
        count+=1
    #print(s)
print(count)

In [ ]:
model_outputs2

In [ ]:
#for model1, model2 in zip(model_outputs,model_outputs2):
 #   print (model1,model2)

In [ ]:
soft_voting=[]
import collections
for model1, model2 in zip(model_outputs,model_outputs2):
    
    #print(soft_voting_x)
    average_zero=max(model1[0],model2[0])
    average_one=max(model1[1],model2[1])
    #print('ávg 0',average_zero, 'avg 1',average_one)
    soft_voting_result=0 if average_zero>average_one else 1
    #print('soft_voting_result',soft_voting_result)
    soft_voting.append(soft_voting_result)
   

In [ ]:
df_en_test['soft_voting_result']=soft_voting


In [ ]:
#soft voting results
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
tn, fp, fn, tp=confusion_matrix(df_en_test['Labels'], df_en_test['soft_voting_result']).ravel()
tn, fp, fn, tp

In [ ]:
p=tp/(tp+fp)
p

In [ ]:
r=tp/(tp+fn)
r

In [ ]:
f1=(2*p*r)/ (p+r)
f1

In [ ]:
precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'])

In [ ]:
precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'], average='binary')

In [ ]:
#precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'], average='macro')

In [ ]:
#precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'], average='weighted')

In [ ]:
#precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'], average='micro')

In [ ]:
df_en_test[(df_en_test['Labels'] == 1) & (df_en_test['soft_voting_result'] == 1)]
df_en_testWrong2=df_en_test[(df_en_test['Labels'] != df_en_test['soft_voting_result'])]
df_en_testWrong2
count=0
for s in df_en_testWrong2['Sentence']:
  #print(s)
  if 'specifically' in s:
        count+=1
print(count)


## Train another way by making a new model

In [ ]:
# Create a ClassificationModel
model_en2 = ClassificationModel(
    #"roberta", 
    #"roberta-base",
    #"xlmroberta", 
    #"xlm-roberta-base",
    #"bert", 
    #"bert-base-cased",
    "electra",
    "google/electra-base-discriminator",
    args=train_args
)
model_en2.train_model(
        df_en_train,
        eval_df=eval_df,
        accuracy=lambda truth, predictions: accuracy_score(
            truth, [round(p) for p in predictions]
        ),
)
#model_en2.train_model(df_en_train, eval_file=eval_df)

In [ ]:
df_en_test

result3, model_outputs3, wrong_predictions3 = model_en2.eval_model(df_en_test, f1=f1_score, r=recall_score, p=precision_score)

result3

In [ ]:
soft_voting=[]
import collections
for model1, model3 in zip(model_outputs,model_outputs3):
    
    #print(soft_voting_x)
    average_zero=(model1[0] +model3[0] *1.2)/2
    average_one=(model1[1] +model3[1]*1.2)/2
    #print('ávg 0',average_zero, 'avg 1',average_one)
    soft_voting_result=0 if average_zero>average_one else 1
    #print('soft_voting_result',soft_voting_result)
    soft_voting.append(soft_voting_result)

In [ ]:
df_en_test['soft_voting_result']=soft_voting


In [ ]:
#soft voting results
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
tn, fp, fn, tp=confusion_matrix(df_en_test['Labels'], df_en_test['soft_voting_result']).ravel()
tn, fp, fn, tp

In [ ]:
precision_recall_fscore_support(df_en_test['Labels'], df_en_test['soft_voting_result'], average='binary')